In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [2]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator

In [3]:
df = pd.read_csv('../datasets/diagnosis.csv')
df.columns=['temperature', 'nausea_occurence', 'lumbar_pain', 'urine_pushing', 
               'micturition_pains', 'bis','inflammation_pred', 'nephritis_pred']
df.to_csv('acute_inflammations.csv', index=False)

In [4]:
df.shape

(120, 8)

In [5]:
df

,temperature,nausea_occurence,lumbar_pain,urine_pushing,micturition_pains,bis,inflammation_pred,nephritis_pred
0,35.5,0,1,0,0,0,0,0
1,35.9,0,0,1,1,1,1,0
2,35.9,0,1,0,0,0,0,0
3,36.0,0,0,1,1,1,1,0
4,36.0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
115,41.4,0,1,1,0,1,0,1
116,41.5,0,0,0,0,0,0,0
117,41.5,1,1,0,1,0,0,1
118,41.5,0,1,1,0,1,0,1


In [6]:
bdf = BaseDatasetTransform(df,target=['inflammation_pred','nephritis_pred'])

In [7]:
df,categorical_features = bdf.fit_transform()

Series([], dtype: int64)

В колонке(ах) ['inflammation_pred', 'nephritis_pred'] нет пропущенных значений
-------------------------------------------
В наборе данных нет пропущенных значений
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index([], dtype='object')

Числовые колонки:
Index(['temperature', 'nausea_occurence', 'lumbar_pain', 'urine_pushing',
       'micturition_pains', 'bis', 'inflammation_pred', 'nephritis_pred'],
      dtype='object')


In [8]:
X_train, X_test, y_train, y_test  = bdf.get_train_test_split()

Количество значений целевой переменной по категориям:
Количество значений комбинаций целевых переменных по категориям:
1_0    40
0_1    31
0_0    30
1_1    19
Name: count, dtype: int64
Следует ли выполнить стратифицированное раздеение на обучающую и тестовую выборку? y/n
Разделение датасета выполнено успешно


In [9]:
from sklearn.multioutput import MultiOutputClassifier


lgbm_model = MultiOutputClassifier(lgb.LGBMClassifier(verbose=-1))

columns_to_delete = []  

unique_classes = len(set(y_train))
print(unique_classes)

model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    #objective='binary',  # Для бинарной классификации
    #metric='binary_error'  # Метрика для бинарной задачи
)

2


In [10]:
metrics = [ 'precision', 'recall', 'f1_macro','f1_samples','hamming_loss']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics)

evaluator.fit(X_train, y_train)

ModelEvaluator(metrics=['precision', 'recall', 'f1_macro', 'f1_samples',
                        'hamming_loss'],
               model_trainer=BaseModelTrain(categorical_features=[],
                                            columns_to_delete=[],
                                            model=MultiOutputClassifier(estimator=LGBMClassifier(verbose=-1))))

In [11]:
evaluator.evaluate_to_dataframe(X_test, y_test)

c:\Users\Dmitriy.Burnashev\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1757: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


,Metric,Score
0,precision,1.00
1,recall,1.00
2,f1,1.00
3,f1_samples,0.75
4,hamming_loss,0.00
